In [1]:
import os
pwd_list = os.getcwd().split('\\')
if pwd_list[-1] != 'RiskAI' and 'RiskAI' in pwd_list:
    os.chdir('/'.join(pwd_list[:pwd_list.index('RiskAI')+1]))

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from preprocess import get_datas

In [4]:
def mapa_calor(df):
    df_corr = df.corr()[cols]
    df_corr = df_corr.loc[[idx for idx in df_corr.index if 
                          idx not in cols]]
    fig = px.imshow(df_corr.T, width=40*len(df_corr), height=600, color_continuous_scale='RdBu', zmin=-1, zmax=1)
    
    for i, col in enumerate(df_corr.columns):
        best_corr = df_corr.iloc[np.argmax(df_corr[col].abs())]
        print(f'MAIOR CORREL: {col} X {best_corr.name}: {best_corr[col]:.4f} ')

    return fig

# Carregamento dos dados
- Dados foram colocados na mesma escala com MinMaxScaler
- Variavel exogena foram avaliado os valores com lag, pois é a informação disponivel para a variavel target 

In [82]:
scaler = MinMaxScaler()
df = get_datas(return_df=True, scaler=scaler)

In [83]:
exog = 'acwi'
cols = ['GSRAII Index', 'GSRAII_diff', 'acwi_log_diff']
df = df[cols]
df.rename(columns={'acwi_log_diff':exog}, inplace=True)
cols = ['GSRAII Index', 'GSRAII_diff', exog]

In [90]:
df.corr()

,GSRAII Index,GSRAII_diff,acwi
GSRAII Index,1.000000,0.176732,0.270787
GSRAII_diff,0.176732,1.000000,0.668366
acwi,0.270787,0.668366,1.000000


In [94]:
fig = px.scatter_matrix(df, width=800, height=800,)
fig.show()

In [86]:
fig = px.line(df,x=range(len(df)),  y=cols, title='')
fig.show()

# Analise lag

In [ ]:
df_lag = df.copy()

for i in range(1, 11):
    df_lag[f'{exog}_lag_{i}'] = df_lag[exog].shift(i).values

In [95]:
mapa_calor(df_lag)

MAIOR CORREL: GSRAII Index X acwi_lag_2: 0.2395 
MAIOR CORREL: GSRAII_diff X acwi_lag_1: -0.1046 
MAIOR CORREL: acwi X acwi_lag_6: 0.0754 


In [101]:
fig = px.scatter_matrix(df_lag[cols+['acwi_lag_1','acwi_lag_2']], width=800, height=800)
fig.show()

# Analise Médias Móveis

In [89]:
df_medmov = df_lag.copy()
cols_lag = [col for col in df_medmov.columns if 'lag' in col]

for lag in cols_lag:
    for i in range(2, 30):
        df_medmov[f'{lag}_mm_{i}'] = df_medmov[f'{lag}'].rolling(i).mean()

mapa_calor(df_medmov.drop(columns=cols_lag))

MAIOR CORREL: GSRAII Index X acwi_lag_1_mm_20: 0.7015 
MAIOR CORREL: GSRAII_diff X acwi_lag_1_mm_5: -0.1167 
MAIOR CORREL: acwi X acwi_lag_10_mm_13: 0.0510 


# Analise desvio padrão

In [44]:
df_std = df_lag.copy()

cols_lag = [col for col in df_std.columns if 'lag' in col]

for lag in cols_lag:
    for i in range(4, 21):
        df_std[f'{lag}_std_{i}'] = df_std[f'{lag}'].rolling(i).mean()

mapa_calor(df_std.drop(columns=cols_lag))

MAIOR CORREL: GSRAII Index X acwi_lag_1_std_20: 0.7015 
MAIOR CORREL: GSRAII_diff X acwi_lag_1_std_5: -0.1167 
MAIOR CORREL: acwi X acwi_lag_10_std_13: 0.0510 


In [ ]:
# diferenca de media, IFR, MACD